# Read Dataset

In [1]:
# Import library yang diperlukan
import os
import random
from sklearn.model_selection import train_test_split

#ignore warning
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Tentukan jalur ke direktori dataset
dataset_dir = 'D:/UTY/Semester 7/Pengembangan Aplikasi AI/Pengenal-Citra-Nomor-Kendaraan/Dataset/K1/'

# Buat daftar semua berkas citra dalam direktori dataset
berkas_citra = [os.path.join(dataset_dir, berkas) for berkas in os.listdir(dataset_dir) if berkas.endswith('.jpg')]

# Tetapkan nilai benih acak untuk konsistensi
random.seed(42)

# Tentukan rasio data untuk pelatihan dan pengujian
rasio_pelatihan = 0.8  # 80% untuk pelatihan, 20% untuk pengujian

# Bagi dataset menjadi set pelatihan dan pengujian
berkas_pelatihan, berkas_pengujian = train_test_split(berkas_citra, train_size=rasio_pelatihan, test_size=1-rasio_pelatihan)

# Buat direktori untuk dataset pelatihan dan pengujian jika belum ada
direktori_pelatihan = os.path.join(dataset_dir, 'pelatihan')
direktori_pengujian = os.path.join(dataset_dir, 'pengujian')
os.makedirs(direktori_pelatihan, exist_ok=True)
os.makedirs(direktori_pengujian, exist_ok=True)

# Pindahkan berkas ke direktori yang sesuai
for berkas in berkas_pelatihan:
    tujuan = os.path.join(direktori_pelatihan, os.path.basename(berkas))
    os.rename(berkas, tujuan)

for berkas in berkas_pengujian:
    tujuan = os.path.join(direktori_pengujian, os.path.basename(berkas))
    os.rename(berkas, tujuan)

# Cetak jumlah citra dalam masing-masing dataset
print(f"Total citra: {len(berkas_citra)}")
print(f"Citra pelatihan: {len(berkas_pelatihan)}")
print(f"Citra pengujian: {len(berkas_pengujian)}")

ValueError: With n_samples=0, test_size=0.19999999999999996 and train_size=0.8, the resulting train set will be empty. Adjust any of the aforementioned parameters.